# Video pipeline with Yolo + Resnet Model

In [1]:
import os
import sys
from pprint import PrettyPrinter
import matplotlib.pyplot as plt
from typing import Dict, List
import numpy as np

pp = PrettyPrinter(indent=4)
from barazmoon.twitter import twitter_workload_generator

# get an absolute path to the directory that contains parent files
__file__ = globals()["_dh"][0]
project_dir = __file__ = globals()["_dh"][0]
sys.path.append(os.path.normpath(os.path.join(project_dir, "..", "..", "..")))

from experiments.utils.constants import FINAL_RESULTS_PATH
from experiments.utils.parser import AdaptationParser
from experiments.utils.drawing import draw_cumulative_with_grouping

In [2]:
metaserieses = [14 ,14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14]
serieses = [1, 2, 3, 5, 21, 22, 23, 25, 41, 42, 43, 45, 61, 62, 63, 65]

series_meta = {
    "video": {"IPA": 1, "FA2-low": 2, "FA2-high": 3, "RIM": 5},
    "audio-qa": {"IPA": 21, "FA2-low": 22, "FA2-high": 23, "RIM": 25},
    "audio-sent": {"video": 41, "audio-qa": 42, "audio-sent": 43, "sum-qa": 45},
    "sum-qa": {"video": 61, "audio-qa": 62, "audio-sent": 63, "sum-qa": 65}
}
# series_names = {
#     1: "IPA",
#     2: "FA2-low",
#     3: "FA2-high",
#     # 403: "RIM-low",
#     5: "RIM-high",
#     21: "IPA",
#     22: "FA2-low",
#     23: "FA2-high",
#     # 403: "RIM-low",
#     25: "RIM-high",
#     41: "IPA",
#     42: "FA2-low",
#     43: "FA2-high",
#     # 403: "RIM-low",
#     45: "RIM-high",
#     61: "IPA",
#     62: "FA2-low",
#     63: "FA2-high",
#     # 403: "RIM-low",
#     65: "RIM-high",
# }

series_paths = {
series: os.path.join(FINAL_RESULTS_PATH, "metaseries", str(metaseries), "series", str(series))
    for series, metaseries in zip(serieses, metaserieses)
}

loaders = {
series: AdaptationParser(
        series_path=series_path, model_name="video", type_of="router_pipeline"
    )
    for series, series_path in series_paths.items()
}

In [3]:
series_paths

{1: '/home/cc/infernece-pipeline-joint-optimization/data/results/final/metaseries/14/series/1',
 2: '/home/cc/infernece-pipeline-joint-optimization/data/results/final/metaseries/14/series/2',
 3: '/home/cc/infernece-pipeline-joint-optimization/data/results/final/metaseries/14/series/3',
 5: '/home/cc/infernece-pipeline-joint-optimization/data/results/final/metaseries/14/series/5',
 21: '/home/cc/infernece-pipeline-joint-optimization/data/results/final/metaseries/14/series/21',
 22: '/home/cc/infernece-pipeline-joint-optimization/data/results/final/metaseries/14/series/22',
 23: '/home/cc/infernece-pipeline-joint-optimization/data/results/final/metaseries/14/series/23',
 25: '/home/cc/infernece-pipeline-joint-optimization/data/results/final/metaseries/14/series/25',
 41: '/home/cc/infernece-pipeline-joint-optimization/data/results/final/metaseries/14/series/41',
 42: '/home/cc/infernece-pipeline-joint-optimization/data/results/final/metaseries/14/series/42',
 43: '/home/cc/infernece-pip

In [4]:
loaders

{1: <experiments.utils.parser.AdaptationParser at 0x7f8fef62d460>,
 2: <experiments.utils.parser.AdaptationParser at 0x7f9098278dc0>,
 3: <experiments.utils.parser.AdaptationParser at 0x7f90982191f0>,
 5: <experiments.utils.parser.AdaptationParser at 0x7f9098278d90>,
 21: <experiments.utils.parser.AdaptationParser at 0x7f9098278f10>,
 22: <experiments.utils.parser.AdaptationParser at 0x7f8fef62d580>,
 23: <experiments.utils.parser.AdaptationParser at 0x7f8fef62d430>,
 25: <experiments.utils.parser.AdaptationParser at 0x7f90982a9610>,
 41: <experiments.utils.parser.AdaptationParser at 0x7f90982a9130>,
 42: <experiments.utils.parser.AdaptationParser at 0x7f8fef388e50>,
 43: <experiments.utils.parser.AdaptationParser at 0x7f8fef64d520>,
 45: <experiments.utils.parser.AdaptationParser at 0x7f90982bf910>,
 61: <experiments.utils.parser.AdaptationParser at 0x7f8fef3886d0>,
 62: <experiments.utils.parser.AdaptationParser at 0x7f8fef64da00>,
 63: <experiments.utils.parser.AdaptationParser at 0

In [5]:
accuracy_methods = {}
adaptation_intervals = {}
simulation_modes = {}
configs = {}
for series, loader in loaders.items():
    configs_exp = loader.load_configs()
    print(f"series: {series} config:\n")
    config = configs_exp["0.yaml"]
    pp.pprint(config)
    configs[series] = config
    accuracy_methods[series] = config["accuracy_method"]
    adaptation_intervals[series] = config["adaptation_interval"]
    simulation_modes[series] = config["simulation_mode"]

series: 1 config:

{   'accuracy_method': 'sum',
    'adaptation_interval': 10,
    'allocation_mode': 'base',
    'alpha': 2,
    'backup_predictor_duration': 2,
    'backup_predictor_type': 'max',
    'baseline_mode': None,
    'batching_cap': 8,
    'benchmark_duration': 1,
    'beta': 1,
    'central_queue': True,
    'debug_mode': False,
    'distrpution_time': 30,
    'drop_limit': 10,
    'gamma': 1e-06,
    'initial_active_model': ['yolov5n', 'resnet18'],
    'initial_batch': [1, 1],
    'initial_cpu_allocation': [1, 1],
    'initial_replica': [1, 1],
    'latency_margin': 0,
    'logs_enabled': False,
    'metadata': 'bursty - ipa - cpu type: compute_cascadelake_r_ib',
    'metaseries': 7,
    'mode': 'exponential',
    'model_name': ['yolo', 'resnet-human'],
    'monitoring_duration': 2,
    'nodes': [   {   'cpu_request': '1',
                     'data_type': 'image',
                     'max_batch_size': '1',
                     'max_batch_time': '1',
                   

In [6]:
# load the sent workload
sent_loads = {}
for series, config in configs.items():
    workload_type = config["workload_type"]
    workload_config = config["workload_config"][0]
    start = workload_config["start"]
    end = workload_config["end"]
    damping_factor = workload_config["damping_factor"]
    sent_loads[series] = twitter_workload_generator(
        days=f"{start}-{end}", damping_factor=damping_factor
    )

In [7]:
# key_config_df = loader.loader.key_config_mapper()
# display(key_config_df)
# key_config_df.columns
results_all = []
simulation_modes

{1: False,
 2: False,
 3: False,
 5: False,
 21: False,
 22: False,
 23: False,
 25: False,
 41: False,
 42: False,
 43: False,
 45: False,
 61: False,
 62: False,
 63: False,
 65: False}

In [8]:
adaptation_logs = dict(
    map(lambda l: (l[0], l[1].load_adaptation_log()), loaders.items())
)

In [9]:
series_changes = {}
for series in serieses:
    series_changes[series] = loaders[series].series_changes(
        adaptation_log=adaptation_logs[series]
    )

## Total core changes

# Real Latencies

In [10]:
timeout_per_second = {}
per_second_results = {}
for series in serieses:
    if not simulation_modes[series]:
        timeout_per_second[series], per_second_results[series] = loaders[
            series
        ].per_second_result_processing()
    else:
        timeout_per_second[series], per_second_results[series] = None, None

In [11]:
metric = "p99"  # [min, max, p99]
metrics_all = {}
for series in serieses:
    metric_columns = list(
        filter(lambda col: metric in col, per_second_results[series].columns)
    )
    metrics_all[series] = per_second_results[series][metric_columns]
    metrics_all[series] = metrics_all[series].to_dict(orient="list")
ylabel = "Second"
data = {}

# TODO

for serie, serie_name in series_names.items():
    data[serie_name] = metrics_all[serie]["e2e_p99"]

In [13]:
data

{'IPA': [0.5556457996368408,
  0.3099641227722168,
  0.3998810076713562,
  0.44612010955810544,
  0.5997866821289063,
  0.2938604545593262,
  0.4001428055763244,
  0.5052165794372558,
  0.5129996395111084,
  0.5032985782623292,
  0.4583745861053467,
  0.48723343849182127,
  0.3695006251335144,
  0.4097952461242676,
  0.6900157570838928,
  4.98299955368042,
  4.308054971694946,
  3.560585074424744,
  0.28940706729888915,
  1.5324681138992309,
  0.7202754092216491,
  4.715174913406371,
  4.859827744960785,
  4.481756753921509,
  4.386256670951843,
  3.94799822807312,
  4.798472595214844,
  4.3814256834983825,
  3.9492090344429016,
  4.11123517036438,
  3.724922409057617,
  3.218627653121948,
  3.1946266531944274,
  2.9784994411468504,
  2.825775306224823,
  2.354230849742889,
  1.8816012477874755,
  1.693999207019806,
  1.2812574124336242,
  1.2156351804733276,
  0.875800552368164,
  1.0082610487937926,
  0.7407375001907348,
  0.9768283247947693,
  0.7246639060974122,
  0.92933030128479,